In [ ]:
# os.chdir(os.path.dirname(__file__))
# print os.getcwd()

%matplotlib inline

import os
import os.path
import numpy

from matplotlib import pyplot

from amuse.community.gadget2.interface import Gadget2
from amuse.datamodel import Particles, AbstractParticleSet
# from amuse.ic.plummer import new_plummer_sphere  # TODO: probably not use Plummer sphere
# from amuse.ic.gasplummer import new_plummer_gas_model
from amuse.ic.kingmodel import new_king_model  # TODO: probably not use King Model
from amuse.ext.galactics_model import new_galactics_model
from amuse.units import units
from amuse.units import constants
from amuse.units import nbody_system
from amuse.units import generic_unit_converter
from amuse.plot import plot
from amuse.plot import native_plot
from amuse.plot import pynbody_column_density_plot
from amuse.plot import HAS_PYNBODY
from amuse.io import read_set_from_file
from amuse.io import write_set_to_file

# sigma = constants.Stefan_hyphen_Boltzmann_constant

# Assumptions:
- Initially two Plummer spheres


# Plummer Sphere
The Plummer sphere may not be the correct initial condition, but we can change this relatively straightforward to a more appropriate particle distribution. The Plummer sphere gives a three dimensional particle density profile of
$$ \rho_{\rm P}(r) = \left( \frac{3M}{4 \pi a^3} \right) \left( 1 + \frac{r^2}{a^2} \right)^{-\frac{5}{2}}, $$ where $M$ is the total cluster mass and $a$ is the Plummer radius (sets size of cluster core). The potential is given by $$ \Phi_{\rm P}(r) = - \frac{GM}{\sqrt{r^2+a^2}} $$

# King Model
If a cluster of stars is equivalent to an isothermal sphere, then the velocities of the stars within the cluster do not depend on their position. A problem arises for an isothermal sphere at the origin, where an infinite density is encoured, thus, an infinite mass. This non-physical situation can be solved by creating a Kind sphere, where at the King radius $r_0$ given by $$ r_0 = \sqrt{\frac{9 \sigma^2}{4 \pi G \rho_0}} $$ the density has dropped to $0.5013 \rho_0$ meaning that $r_0$ is the core radius (defined as the radius where the density is half the central density). 

http://www.astro.caltech.edu/~george/ay20/eaa-starclus.pdf

In [ ]:
def make_clusters():
    print 'Creating two clusters...'
# File 'disk_galactICs.amuse' contains 30k particles, (10k disk, no bulge, 20k halo)
    if os.path.exists('disk_clusterICs.amuse'):
        print 'Galaxy particle set exists.'
        cluster1 = read_set_from_file('disk_clusterICs.amuse', 'amuse') 
    else:
        print 'Creating galaxy1 particle set...'
        halo_number_of_particles = 20000
        converter = generic_unit_converter.ConvertBetweenGenericAndSiUnits(
            1.0e15 | units.MSun, 8.0 | units.Mpc, constants.G)
        cluster1 = new_galactics_model(halo_number_of_particles, disk_number_of_particles=10000,
            generate_bulge_flag=False, do_scale=False, unit_system_converter=converter)
        write_set_to_file(cluster1, 'disk_clusterICs.amuse', 'amuse')

    # cluster1 = converter.to_si(cluster1)
    cluster2 = Particles(len(cluster1))
    cluster2.mass = cluster1.mass
    cluster2.position = cluster1.position
    cluster2.velocity = cluster1.velocity

    # TODO: change cluster position / loop over different initial cluster positions
    cluster1.rotate(0.0, numpy.pi/4, 0.0)
    cluster2.rotate(numpy.pi/6, 0.0, 0.0)
    cluster1.position += [1000.0, 0, 0] | units.Mpc
    cluster2.position -= [1000.0, 0, 0] | units.Mpc
    cluster1.velocity += [0.0, 50.0, 0] | units.km/units.s
    cluster2.velocity -= [0.0, 50.0, 0] | units.km/units.s
    return cluster1, cluster2

In [ ]:
def simulate_merger(cluster1, cluster2):
    converter = nbody_system.nbody_to_si(1.0e15 | units.MSun, 100 | units.Mpc)
    dynamics = Gadget2(converter, number_of_workers=4)
    dynamics.parameters.epsilon_squared = 0.0000001 | nbody_system.length**2
    set1 = dynamics.particles.add_particles(cluster1)
    set2 = dynamics.particles.add_particles(cluster2)
    
    print type(set1)
    print type(set2)
    return
    
    clusters_without_halo = set1[:10000] + set2[:10000]
    # make_plots(dynamics.particles, clusters_without_halo)
    write_set_to_file(dynamics.particles, 'particles_at_time_0.amuse', 'amuse')

    for i in range(1, 101):
        dynamics.evolve_model(i * (25 | units.Myr))
        t = dynamics.model_time.value_in(units.Myr)
        # make_plots(dynamics.particles, clusters_without_halo)
        write_set_to_file(dynamics.particles, 'particles_at_time_{0}.amuse'.format(t), 'amuse')

    dynamics.stop()

In [ ]:
# https://jakevdp.github.io/blog/2013/05/12/embedding-matplotlib-animations/
from tempfile import NamedTemporaryFile

from matplotlib import animation
from IPython.display import HTML

VIDEO_TAG = """<video controls>
<source src="data:video/x-m4v;base64,{0}" type="video/mp4">
Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4', delete=False) as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    pyplot.close(anim._fig)
    return HTML(anim_to_html(anim))

In [ ]:
def make_plots(all_particles, disk_only):
    for j, particles in enumerate([all_particles, disk_only]):
        if HAS_PYNBODY:
            temp_particles = particles.copy()
            temp_particles.u = 1 | units.ms**2
            temp = Gadget2()
            temp.gas_particles.add_particles(temp_particles)
            temp.commit_particles()
            pynbody_column_density_plot(temp.gas_particles, width=300|units.Mpc, units = 'm_p cm^-2', vmin=17, vmax=23)
            temp.stop()
        else:
            native_plot.gca().set_aspect("equal")
            native_plot.xlim(-150, 150)
            native_plot.ylim(-150, 150)
            plot(particles.x.as_quantity_in(units.Mpc), particles.y.as_quantity_in(units.Mpc), 'r.')

        # native_plot.savefig(os.path.join("plots", "plot_galaxy_merger_{0}_{1:=04}.png".format("disk" if j else "total", i)))
        # native_plot.close()
        # native_plot.show()

In [ ]:
cluster1, cluster2 = make_clusters()

temp_particles = Particles()
set1 = temp_particles.add_particles(cluster1)
set2 = temp_particles.add_particles(cluster2)
clusters_without_halo = set1[:10000] + set2[:10000]
import glob
filenames_unsorted = glob.glob('./particles_at_time_*.amuse')
filenames_sorted = sorted(filenames_unsorted, key = lambda x: float(x.split("_")[-1].split(".")[0]))
times = [float(x.split("_")[-1].split(".")[0]) for x in filenames_sorted]

# First set up the figure, the axis, and the plot element we want to animate
fig = pyplot.figure()
ax = pyplot.axes()#xlim=(0, 2), ylim=(-2, 2))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    # native_plot.gca().set_aspect("equal")
    # native_plot.xlim(-150, 150)
    # native_plot.ylim(-150, 150)
    line.set_data([], [])
    return line,

def animate(i):
    fname = filenames_sorted[i]
    time = times[i]
    # print fname, 'at time', time | units.Myr
    particles = read_set_from_file(fname, 'amuse') 
    x, y = particles.x.value_in(units.Mpc), particles.y.value_in(units.Mpc)

    # print i, x[0], y[0]

    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(times), interval=20, blit=True)

# call our new function to display the animation
display_animation(anim)

In [ ]:
help(units)

In [ ]:
import os
print os.getcwd()
print os.path.exists('disk_clusterICs.amuse')

In [ ]:
help(Particles.add_particles)

In [ ]:
"""
In this script we simulate a 1d linear wave in a 2d field,
the periodic boundaries are not handled in the code but by amuse
(much slower at time of writing)
"""
import numpy
import math

from amuse.support.core import late
from amuse.units.quantities import VectorQuantity
from amuse.units.generic_unit_system import *

from amuse.community.capreole.interface import Capreole
from amuse import io
from amuse.io import text

from amuse.datamodel import Grid
from amuse.datamodel.grids import SamplePointsOnMultipleGrids
from amuse.datamodel.grids import SamplePointWithIntepolation
from amuse.datamodel.grids import SamplePointOnCellCenter
try:
    from amuse import plot
    from matplotlib import pyplot
    from matplotlib import animation
    IS_PLOT_AVAILABLE = True
except ImportError:
    IS_PLOT_AVAILABLE = False

USE_BOUNDARIES = True

class EvolveHydrodynamicsCodeWithAmusePeriodicBoundaries(object):

    def __init__(self, code, number_of_grid_points, nghost):
        self.code = code
        self.number_of_grid_points = number_of_grid_points
        self.nghost = nghost

    def set_parameters(self):
        self.code.parameters.x_boundary_conditions = ("interface","interface")
        self.code.parameters.y_boundary_conditions = ("interface","interface")
        self.code.stopping_conditions.number_of_steps_detection.enable()

    def init_channels(self):
        instance = self.code
        self.xbound1 = instance.get_boundary_grid('xbound1')
        self.xbound2 = instance.get_boundary_grid('xbound2')
        self.ybound1 = instance.get_boundary_grid('ybound1')
        self.ybound2 = instance.get_boundary_grid('ybound2')

        self.xbound1_channel = instance.grid[self.number_of_grid_points - self.nghost:,..., ...].new_channel_to(self.xbound1)
        self.xbound2_channel = instance.grid[0: self.nghost,..., ...].new_channel_to(self.xbound2)
        self.ybound1_channel = instance.grid[...,self.number_of_grid_points-self.nghost:, ...].new_channel_to(self.ybound1[self.nghost:self.number_of_grid_points+self.nghost,...,...])
        self.ybound2_channel = instance.grid[...,0:self.nghost, ...].new_channel_to(self.ybound2[self.nghost:self.number_of_grid_points+self.nghost,...,...])

        xbound1_bottom = self.xbound1[0:self.nghost,0:self.nghost,...]
        xbound1_top = self.xbound1[0:self.nghost,self.number_of_grid_points - self.nghost:,...]
        ybound1_left = self.ybound1[0:self.nghost,0:self.nghost,...]
        ybound2_left = self.ybound2[0:self.nghost,0:self.nghost,...]

        self.xbound1_ybound1_channel = xbound1_top.new_channel_to(ybound1_left)
        self.xbound1_ybound2_channel = xbound1_bottom.new_channel_to(ybound2_left)

        xbound2_bottom = self.xbound2[0:self.nghost,0:self.nghost,...]
        xbound2_top = self.xbound2[0:self.nghost,self.number_of_grid_points - self.nghost:,...]
        ybound1_right = self.ybound1[self.number_of_grid_points+self.nghost:,0:self.nghost,...]
        ybound2_right= self.ybound2[self.number_of_grid_points+self.nghost:,0:self.nghost,...]

        self.xbound2_ybound1_channel = xbound2_top.new_channel_to(ybound1_right)
        self.xbound2_ybound2_channel = xbound2_bottom.new_channel_to(ybound2_right)

        self.copy_to_boundary_cells()

    def copy_to_boundary_cells(self):
        self.xbound1_channel.copy()
        self.xbound2_channel.copy()
        self.ybound1_channel.copy()
        self.ybound2_channel.copy()

        self.xbound1_ybound1_channel.copy()
        self.xbound1_ybound2_channel.copy()
        self.xbound2_ybound1_channel.copy()
        self.xbound2_ybound2_channel.copy()

    def evolve_model(self, time, endtime):
        while self.code.model_time < time:

            if (self.code.get_timestep() + self.code.model_time) >= time and time == endtime:
                self.code.parameters.must_evolve_to_exact_time = True

            self.code.evolve_model(time)
            self.copy_to_boundary_cells()

class EvolveHydrodynamicsCodeWithAmusePeriodicBoundariesAndNCodes(object):

    def __init__(self, codes, number_of_grid_points, number_of_grid_points_per_code, nghost):
        self.codes = codes
        self.number_of_grid_points = number_of_grid_points
        self.number_of_grid_points_per_code = number_of_grid_points_per_code
        self.nghost = nghost
        self.min_timestep = None

    def set_parameters(self):
        for code in self.codes:
            code.parameters.x_boundary_conditions = ("interface","interface")
            code.parameters.y_boundary_conditions = ("interface","interface")
            code.stopping_conditions.number_of_steps_detection.enable()

    def init_channels(self):
        channels = []
        after_channels = []
        for index in range(0, len(self.codes)):
            instance = self.codes[index]
            nx,ny,nz = instance.grid.shape
            print nx,ny,nz
            xbound1 = instance.get_boundary_grid('xbound1')
            xbound2 = instance.get_boundary_grid('xbound2')
            xbound1_nghost_x,_,ybound1_nghost_z = xbound1.shape
            xbound2_nghost_x,_,ybound2_nghost_z = xbound2.shape
            print xbound1_nghost_x,xbound2_nghost_x
            ybound1 = instance.get_boundary_grid('ybound1')
            ybound2 = instance.get_boundary_grid('ybound2')
            ybound1_nghost_x,ybound1_nghost_y,ybound1_nghost_z = ybound1.shape
            ybound2_nghost_x,ybound2_nghost_y,ybound2_nghost_z = ybound2.shape
            ybound1_nghost_x = (ybound1_nghost_x - nx) / 2
            ybound2_nghost_x = (ybound2_nghost_x - nx) / 2


            xbound1_channel = instance.grid[nx - xbound1_nghost_x:,..., ...].new_channel_to(xbound1)
            xbound2_channel = instance.grid[0:xbound2_nghost_x,..., ...].new_channel_to(xbound2)
            if index == 0:
                instance11 = self.codes[-1]
            else:
                instance11 = self.codes[index-1]
            print ybound1[ybound1_nghost_x:nx+ybound1_nghost_x,...,...].shape
            print instance11.grid[...,ny-ybound1_nghost_y:, ...].shape
            ybound1_channel = instance11.grid[...,ny-ybound1_nghost_y:, ...].new_channel_to(ybound1[ybound1_nghost_x:nx+ybound1_nghost_x,...,...])
            nx11,ny11,nz11 = instance11.grid.shape
            ybound1_left_channel = instance11.grid[0:ybound1_nghost_x,ny11-ybound1_nghost_y:, ...].new_channel_to(ybound1[0:ybound1_nghost_x,...,...])
            ybound1_right_channel = instance11.grid[nx11-ybound1_nghost_x:,ny11-ybound1_nghost_y:, ...].new_channel_to(ybound1[nx+ybound1_nghost_x:,...,...])

            if index == len(self.codes)-1:
                instance12 = self.codes[0]
            else:
                instance12 = self.codes[index+1]
            ybound2_channel = instance12.grid[...,0:ybound2_nghost_y, ...].new_channel_to(ybound2[ybound2_nghost_x:nx+ybound2_nghost_x,...,...])
            nx12,ny12,nz12 = instance12.grid.shape

            ybound2_left_channel = instance12.grid[0:ybound2_nghost_x,0:ybound2_nghost_y, ...].new_channel_to(ybound2[0:ybound2_nghost_x,...,...])
            ybound2_right_channel = instance12.grid[nx12-ybound2_nghost_x:,0:ybound2_nghost_y, ...].new_channel_to(ybound2[nx+ybound2_nghost_x:,...,...])

            channels.append(xbound1_channel)
            channels.append(xbound2_channel)
            channels.append(ybound1_channel)
            channels.append(ybound2_channel)
            after_channels.append(ybound1_left_channel)
            after_channels.append(ybound1_right_channel)
            after_channels.append(ybound2_left_channel)
            after_channels.append(ybound2_right_channel)

        self.channels = channels
        self.channels.extend(after_channels)
        self.copy_to_boundary_cells()

    def copy_to_boundary_cells(self):
        for channel in self.channels:
            channel.copy()

    def evolve_model(self, time, endtime):

        code = self.codes[0]
        t_unit = code.get_timestep().unit
        while code.model_time < time:
            timesteps = [] | t_unit
            for x in self.codes:
                timesteps.append(x.get_timestep())
            min_timestep = timesteps.min()
            if code.model_time == 0.0 | t_unit:
                min_timestep = time

            if (min_timestep + code.model_time) >= time and time == endtime:
                for x in self.codes:
                    x.parameters.must_evolve_to_exact_time = True
            print min_timestep
            for x in self.codes:
                x.set_timestep(min_timestep)

            for x in self.codes:
                x.evolve_model(x.model_time + (min_timestep *2))
                print "MODEL_TIME:", x.model_time

            self.copy_to_boundary_cells()

class EvolveHydrodynamicsCodeWithAmusePeriodicBoundariesAndNCodesWithDifferentGridsSizes(object):

    """
        first version, grids connect on the y axis, whole system is periodic
    """
    def __init__(self, codes, number_of_grid_points, number_of_grid_points_per_code, nghost):
        self.codes = codes
        self.number_of_grid_points = number_of_grid_points
        self.number_of_grid_points_per_code = number_of_grid_points_per_code
        self.nghost = nghost
        self.min_timestep = None

    def set_parameters(self):
        for code in self.codes:
            code.parameters.x_boundary_conditions = ("interface","interface")
            code.parameters.y_boundary_conditions = ("interface","interface")
            code.stopping_conditions.number_of_steps_detection.enable()

    def init_channels(self):
        channels = []
        after_channels = []
        for index in range(0, len(self.codes)):
            instance = self.codes[index]
            xbound1 = instance.get_boundary_grid('xbound1')
            xbound2 = instance.get_boundary_grid('xbound2')

            ybound1 = instance.get_boundary_grid('ybound1')
            ybound2 = instance.get_boundary_grid('ybound2')

            xbound1_bottom = xbound1[0:self.nghost,0:self.nghost,...]
            xbound1_top = xbound1[0:self.nghost,self.number_of_grid_points - self.nghost:,...]

            xbound1_channel = instance.grid[self.number_of_grid_points - self.nghost:,..., ...].new_channel_to(xbound1)
            xbound2_channel = instance.grid[0: self.nghost,..., ...].new_channel_to(xbound2)

            if index == 0:
                instance11 = self.codes[-1]
            else:
                instance11 = self.codes[index-1]
            ybound1_channel = instance11.grid[...,self.number_of_grid_points_per_code-self.nghost:, ...].new_channel_to(ybound1[self.nghost:self.number_of_grid_points+self.nghost,...,...])
            ybound1_left_channel = instance11.grid[0:self.nghost,self.number_of_grid_points_per_code-self.nghost:, ...].new_channel_to(ybound1[0:self.nghost,...,...])
            ybound1_right_channel = instance11.grid[-self.nghost:,self.number_of_grid_points_per_code-self.nghost:, ...].new_channel_to(ybound1[self.number_of_grid_points+self.nghost:,...,...])

            if index == len(self.codes)-1:
                instance12 = self.codes[0]
            else:
                instance12 = self.codes[index+1]
            ybound2_channel = instance12.grid[...,0:self.nghost, ...].new_channel_to(ybound2[self.nghost:self.number_of_grid_points+self.nghost,...,...])

            xbound1next = instance12.get_boundary_grid('xbound1')
            xbound2next = instance12.get_boundary_grid('xbound2')
            ybound2_left_channel = xbound1next[...,0:self.nghost, ...].new_channel_to(ybound2[0:self.nghost,...,...])
            ybound2_right_channel = xbound2next[...,0:self.nghost, ...].new_channel_to(ybound2[self.number_of_grid_points+self.nghost:,...,...])

            channels.append(xbound1_channel)
            channels.append(xbound2_channel)
            channels.append(ybound1_channel)
            channels.append(ybound2_channel)
            after_channels.append(ybound1_left_channel)
            after_channels.append(ybound1_right_channel)
            after_channels.append(ybound2_left_channel)
            after_channels.append(ybound2_right_channel)

        self.channels = channels
        self.channels.extend(after_channels)
        self.copy_to_boundary_cells()

    def copy_to_boundary_cells(self):
        for channel in self.channels:
            channel.copy()

    def evolve_model(self, time, endtime):

        code = self.codes[0]
        t_unit = code.get_timestep().unit
        while code.model_time < time:
            timesteps = [] | t_unit
            for x in self.codes:
                timesteps.append(x.get_timestep())
            min_timestep = timesteps.min()
            if code.model_time == 0.0 | t_unit:
                min_timestep = time

            if (min_timestep + code.model_time) >= time and time == endtime:
                for x in self.codes:
                    x.parameters.must_evolve_to_exact_time = True

            for x in self.codes:
                x.set_timestep(min_timestep)

            for x in self.codes:
                x.evolve_model(time)

            self.copy_to_boundary_cells()



class EvolveHydrodynamicsCodeWithPeriodicBoundaries(object):
    def __init__(self, code):
        self.code = code

    def set_parameters(self):
        self.code.parameters.x_boundary_conditions = ("periodic","periodic")
        self.code.parameters.y_boundary_conditions = ("periodic","periodic")
        self.code.stopping_conditions.number_of_steps_detection.disable()

    def init_channels(self):
        pass

    def evolve_model(self, time, endtime):
        while self.code.model_time < time:

            if (self.code.get_timestep() + self.code.model_time) >= time and time == endtime:
                self.code.parameters.must_evolve_to_exact_time = True

            self.code.evolve_model(time)


class CalculateLinearWave1D(object):
    gamma = 5.0/3.0
    wave_flag = 0

    def __init__(self,
        number_of_grid_points =  10,
        number_of_workers = 1,
        name_of_the_code = "athena",
        amplitude = 1e-6 | speed,
        vflow_factor = 1.0,
        grid_length = 1.0 | length,
        number_of_steps = 10,
        use_boundaries = True,
        number_of_codes = 2
    ):

        self.number_of_grid_points = number_of_grid_points
        self.number_of_workers = number_of_workers
        self.name_of_the_code = name_of_the_code
        self.amplitude = amplitude
        self.vflow_factor = vflow_factor
        self.grid_length = 1.0 | length
        self.number_of_steps = number_of_steps
        self.number_of_codes = number_of_codes
        self.dimensions_of_mesh = (
            self.number_of_grid_points,
            self.number_of_grid_points,
            4
        )
        self.nghost = 4
        self.use_boundaries = use_boundaries

    def new_instance_of_code(self):
        if 1:
            if self.name_of_the_code == 'athena':
                self.name_of_the_code = 'capreole'
            else:
                self.name_of_the_code = 'athena'

        attribute = "new_instance_of_{0}_code".format(self.name_of_the_code.lower())
        return getattr(self,attribute)()

    def new_instance_of_capreole_code(self):
        result=Capreole(number_of_workers=self.number_of_workers)
        self.dimensions_of_mesh =  (
            self.number_of_grid_points,
            self.number_of_grid_points,
            4
        )
        self.nghost = 2
        return result

    def new_instance_of_athena_code(self):
        from amuse.community.athena.interface import Athena
        result=Athena(number_of_workers=self.number_of_workers)
        result.parameters.gamma = self.gamma
        result.parameters.courant_number=0.4
        self.nghost = 4
        return result


    def new_instance_of_mpiamrvac_code(self):
        raise Exception("MPIAMRVAC does not yet have support for detailed boundaries in amuse")
        from amuse.community.mpiamrvac.interface import MpiAmrVac
        result=MpiAmrVac(mode="2d", number_of_workers=self.number_of_workers, debugger="xterm")
        result.set_parameters_filename(result.default_parameters_filename)
        result.initialize_code()
        return result

    def set_parameters(self, codes, evolve):

        mesh_for_code = list(self.dimensions_of_mesh)
        mesh_for_code[1] /= self.number_of_codes
        for instance in codes:
            instance.parameters.mesh_size = list(mesh_for_code)

            instance.parameters.length_x = self.grid_length
            instance.parameters.length_y = self.grid_length / self.number_of_codes
            instance.parameters.length_z = self.grid_length

            instance.parameters.z_boundary_conditions = ("periodic","periodic")
        evolve.set_parameters()

        for instance in codes:
            instance.commit_parameters()

    def new_grid(self):
        grid = Grid.create(self.dimensions_of_mesh, [1,1,1] | length)
        self.clear_grid(grid)
        return grid

    def clear_grid(self, grid):
        density = mass / length**3
        momentum =  speed * density
        energy =  mass / (time**2 * length)

        grid.rho =  0.0 | density
        grid.rhovx = 0.0 | momentum
        grid.rhovy = 0.0 | momentum
        grid.rhovz = 0.0 | momentum
        grid.energy = 0.0 | energy

        return grid


    def new_rhoe_right_eigenmatrix(self, velocity, amplitude, enthalpy):
        right_eigenmatrix = numpy.zeros ( (5,5) ) | speed

        right_eigenmatrix[0][0] = 1.0 | speed;
        right_eigenmatrix[1][0] = velocity[0] - amplitude;
        right_eigenmatrix[2][0] = velocity[1];
        right_eigenmatrix[3][0] = velocity[2];
        right_eigenmatrix[4][0] = (1.0 | time/length) * (enthalpy - velocity[0]*amplitude);
        #right_eigenmatrix[0][1] = 0.0;
        #right_eigenmatrix[1][1] = 0.0;
        right_eigenmatrix[2][1] = 1.0 | speed;
        #right_eigenmatrix[3][1] = 0.0;
        right_eigenmatrix[4][1] = velocity[1];

        #right_eigenmatrix[0][2] = 0.0; */
        #right_eigenmatrix[1][2] = 0.0; */
        #right_eigenmatrix[2][2] = 0.0; */
        right_eigenmatrix[3][2] = 1.0 | speed;
        right_eigenmatrix[4][2] = velocity[2];

        right_eigenmatrix[0][3] = 1.0 | speed;
        right_eigenmatrix[1][3] = velocity[0];
        right_eigenmatrix[2][3] = velocity[1];
        right_eigenmatrix[3][3] = velocity[2];
        right_eigenmatrix[4][3] = 0.5*velocity.length();

        right_eigenmatrix[0][4] = 1.0 | speed;
        right_eigenmatrix[1][4] = velocity[0] + amplitude;
        right_eigenmatrix[2][4] = velocity[1];
        right_eigenmatrix[3][4] = velocity[2];
        right_eigenmatrix[4][4] = (1.0 | time/length) *  (enthalpy + velocity[0]*amplitude);
        return right_eigenmatrix

    def initialize_grid(self, grid):
        density = mass / length**3
        momentum =  speed * density
        energy =  mass / (time**2 * length)
        rho =  1.0 | density
        pressure = (1.0/self.gamma) | (mass / (length * time**2))
        vx = (self.gamma * pressure / rho).sqrt()
        velocity = self.vflow_factor * vx * [1.0, 0.0, 0.0]
        velocity_squared = velocity.length()
        energy = (pressure/(self.gamma - 1.0) + (0.5 | length / time )*rho*velocity_squared)
        enthalpy = (energy + pressure)/rho;
        amplitude_squared = (self.gamma - 1.0) * max(enthalpy - (0.5 | length/time)* velocity_squared, 1e-100 | enthalpy.unit)
        amplitude =amplitude_squared.sqrt()

        nwave = 5
        eigenvalues = ([0] * nwave) | speed
        eigenvalues[0] = velocity[0] - amplitude
        eigenvalues[1] = velocity[0]
        eigenvalues[2] = velocity[0]
        eigenvalues[3] = velocity[0]
        eigenvalues[4] = velocity[0] + amplitude

        right_eigenmatrix = self.new_rhoe_right_eigenmatrix(velocity, amplitude, enthalpy)

        grid.rho = rho
        grid.energy = energy
        grid.rhovy = rho*self.vflow_factor*(1.0 | speed)

        wave = self.amplitude*numpy.sin(grid.y * (2.0 | length**-1)*numpy.pi)

        grid.rho += wave*right_eigenmatrix[0][self.wave_flag] * (1.0 |mass * time**2 / length**5)
        grid.rhovx += wave*right_eigenmatrix[3][self.wave_flag] * (1.0 |mass * time / length**4)
        grid.rhovy += wave*right_eigenmatrix[1][self.wave_flag] * (1.0 |mass * time / length**4)
        grid.rhovz += wave*right_eigenmatrix[2][self.wave_flag] * (1.0 |mass * time / length**4)
        grid.energy += wave*right_eigenmatrix[4][self.wave_flag] *(1.0 | mass  / length**3)

    def store_grids(self, grids, step):
        if __name__ == '__plot__':
            return

        grids_in_memory = [x.copy() for x in grids]
        io.write_set_to_file(
            grids_in_memory,
            "linear_wave_{2}_{0}_{1}.vtu".format(self.number_of_grid_points, step, self.name_of_the_code),
            "vtu",
            is_multiple=True
        )

    def new_evolve_object(self, instance):
        """Returns a special object to evolve to code in time"""
        if self.use_boundaries:
            #return EvolveHydrodynamicsCodeWithAmusePeriodicBoundaries(instance[0], self.number_of_grid_points, self.nghost)
            return EvolveHydrodynamicsCodeWithAmusePeriodicBoundariesAndNCodes(instance, self.number_of_grid_points, self.number_of_grid_points / self.number_of_codes , self.nghost)
        else:
            return EvolveHydrodynamicsCodeWithPeriodicBoundaries(instance[0])

    def initialize(self):
        self.codes = []
        for i in range(self.number_of_codes):
            self.codes.append(self.new_instance_of_code())

        self.evolve = self.new_evolve_object(self.codes)

        self.set_parameters(self.codes, self.evolve)

        self.start_grids = []

        offset = 0.0 * self.grid_length
        for code in self.codes:
            for x in code.itergrids():
                inmem = x.copy()
                self.clear_grid(inmem)
                inmem.y += offset
                self.initialize_grid(inmem)
                self.start_grids.append(inmem)
                from_model_to_code = inmem.new_channel_to(x)
                from_model_to_code.copy()
            offset += self.grid_length / self.number_of_codes


        for x in self.codes:
            x.initialize_grid()
        self.evolve.init_channels()

    def evolve_model(self,time, endtime):
        for code in self.codes:
            code.parameters.must_evolve_to_exact_time = False
        self.evolve.evolve_model(time, endtime)


    def get_grids(self):
        result = []

        offset = 0.0 * self.grid_length
        for code in self.codes:
            for x in code.itergrids():
                inmem = x.copy()
                inmem.y += offset
                result.append(inmem)
            offset += self.grid_length / self.number_of_codes

        return result

    def stop(self):
        print "terminating code"

        for code in self.codes:
            code.stop()

# import sys
def main():
    number_of_grid_points = 60
    name_of_the_code = 'athena'
    number_of_steps = 2000
    vflow_factor = -1.0
    pertubation_amplitude = 1e-4 | speed
    grid_length = 1.0 | length
    # number_of_codes = int(sys.argv[1])
    number_of_codes = int(1)
    if number_of_grid_points % number_of_codes != 0:
        raise Exception("grid points should be dividable by the number of codes")
    model1 = CalculateLinearWave1D(
        number_of_grid_points = number_of_grid_points,
        number_of_workers = 1,
        name_of_the_code = name_of_the_code,
        amplitude = pertubation_amplitude,
        vflow_factor = vflow_factor,
        grid_length = grid_length,
        number_of_steps = number_of_steps,
        use_boundaries = True,
        number_of_codes = number_of_codes
    )
    if 0:
        model2 = CalculateLinearWave1D(
            number_of_grid_points = number_of_grid_points,
            number_of_workers = 1,
            name_of_the_code = name_of_the_code,
            amplitude = pertubation_amplitude,
            vflow_factor = vflow_factor,
            grid_length = grid_length,
            number_of_steps = number_of_steps,
            use_boundaries = False,
            number_of_codes = number_of_codes
        )

    if not IS_PLOT_AVAILABLE:
        print "Plot is not available. stop now"
        return
    model1.initialize()
    #model2.initialize()


    grids1 = model1.get_grids()
    #grids2 = model2.get_grids()


    figure = pyplot.figure(figsize=(10,5))
    plot1 = figure.add_subplot(1,1,1)
    lines = []
    ys = []
    for grid in grids1:
        y = grid.y[0,...,0].value_in(length)
        ys.append(y)
        rho = grid.rho[0,...,0].value_in(density)
        line = plot1.plot(y,rho)[0]
        lines.append(line)

    end_time = 10.0 | time
    dt = end_time / number_of_steps

    t = dt
    step = 1

    title = figure.suptitle('{0:.3f}'.format(0.0))
    variables = [t, step]

    def update(xx):
        t, step = variables
        title.set_text('{0:.3f}'.format(t.value_in(time)))
        model1.evolve_model(t, end_time)
        #model2.evolve_model(t, end_time)
        t += dt
        grids1 = model1.get_grids()
        #grids2 = model2.get_grids()

        for line, grid in zip(lines, grids1):
            y = grid.y[0,...,0].value_in(length)
            rho = grid.rho[0,...,0].value_in(density)
            line.set_data(y,rho)
            #line = plot1.plot(y,rho)[0]
            #lines.append(line)
        print t
        step += 1
        variables[0] = t
        variables[1] = step
        return lines
    if 1:
        process = animation.FuncAnimation(
            figure,
            update,
            numpy.arange(1, 200),
            interval=25,
            blit=False
        )
    else:
        update(0)
        update(0)
        pass


    pyplot.show()
    model1.stop()
    #model2.stop()

if __name__ == "__main__":
    main()

In [3]:
%matplotlib inline

In [4]:
import os.path
from amuse.test.amusetest import get_path_to_results
try:
    from matplotlib import pyplot
    HAS_MATPLOTLIB = True
    from amuse.plot import plot, semilogy, xlabel, ylabel, loglog
except ImportError:
    HAS_MATPLOTLIB = False

from amuse.units import units
from amuse.units import constants
from amuse.units.generic_unit_converter import ConvertBetweenGenericAndSiUnits
from amuse.support.exceptions import AmuseException
from amuse.community.mesa.interface import MESA
from amuse.community.gadget2.interface import Gadget2
from amuse.community.fi.interface import Fi
from amuse.ext.star_to_sph import convert_stellar_model_to_SPH

import numpy

from amuse.datamodel import Particles
from amuse.datamodel import ParticlesSuperset
from amuse.datamodel import Grid
def head_on_stellar_merger(
        masses = [0.3, 3.0] | units.MSun,
        star_age = 310.0 | units.Myr,
        initial_separation = 4.0 | units.RSun,
        angle = numpy.pi / 3,
        initial_speed = 3000.0 | units.km / units.s,
        initial_speed_perpendicular = 30.0 | units.km / units.s,
        number_of_sph_particles = 50000,
        t_end = 1.0e4 | units.s,
        sph_code = Fi,
    ):
    """
    masses: Mass of the two stars
    star_age: Initial age of the stars
    number_of_sph_particles: Total number of particles of both stars, divided according to their masses
    t_end: (Physical, not computational) duration of the hydrodynamics simulation
    sph_code: Code to use for the hydrodynamics simulation
    """

    # Convert some of the input parameters to string, for use in output file names:
    n_string = "n" + ("%1.0e"%(number_of_sph_particles)).replace("+0","").replace("+","")
    t_end_string = "t" + ("%1.0e"%(t_end.value_in(units.s))).replace("+0","").replace("+","")

    base_output_file_name = os.path.join(get_path_to_results(), "stellar_merger_"+n_string+"_"+t_end_string)

    stars =  Particles(2)
    stars.mass = masses
    try:
        stellar_evolution = MESA(redirection='none', number_of_workers=4)
        stellar_evolution.initialize_code()
    except:
        print "MESA was not built. Returning."
        return
    stellar_evolution.commit_parameters()
    stellar_evolution.particles.add_particles(stars)
    stellar_evolution.commit_particles()
    print "Evolving stars with MESA..."
    stellar_evolution.evolve_model(star_age)

    number_of_sph_particles_1 = int(round(number_of_sph_particles *
        (stellar_evolution.particles[0].mass / stellar_evolution.particles.mass.sum())))
    number_of_sph_particles_2 = number_of_sph_particles - number_of_sph_particles_1
    print "Creating initial conditions from a MESA stellar evolution model:"
    print stellar_evolution.particles[0].mass, "star consisting of", number_of_sph_particles_1, "particles."
    sph_particles_1 = convert_stellar_model_to_SPH(
        stellar_evolution.particles[0],
        number_of_sph_particles_1,
        seed=12345
    ).gas_particles
    print stellar_evolution.particles[1].mass, "star consisting of", number_of_sph_particles_2, "particles."
    sph_particles_2 = convert_stellar_model_to_SPH(
        stellar_evolution.particles[1],
        number_of_sph_particles_2
    ).gas_particles

    initial_separation += stellar_evolution.particles.radius.sum()
    sph_particles_2.x  += numpy.cos(angle) * initial_separation
    sph_particles_2.y  += numpy.sin(angle) * initial_separation
    sph_particles_1.vx += numpy.cos(angle) * initial_speed - numpy.sin(angle) * initial_speed_perpendicular
    sph_particles_1.vy += numpy.cos(angle) * initial_speed_perpendicular + numpy.sin(angle) * initial_speed
    view = [-0.5, 0.5, -0.5, 0.5] * (initial_separation + stellar_evolution.particles.radius.sum())
    stellar_evolution.stop()

    all_sph_particles = ParticlesSuperset([sph_particles_1, sph_particles_2])
    all_sph_particles.move_to_center()

    unit_converter = ConvertBetweenGenericAndSiUnits(1.0 | units.RSun, constants.G, t_end)
    hydro_legacy_code = sph_code(unit_converter, redirection='none', number_of_workers=4)
    n_steps = 100
    hydro_legacy_code.parameters.n_smooth = 96
    try:
        hydro_legacy_code.parameters.timestep = t_end / n_steps
    except Exception as exc:
        if not "parameter is read-only" in str(exc): raise
    hydro_legacy_code.gas_particles.add_particles(all_sph_particles)

    print "Evolving to t =", t_end, " (using", sph_code.__name__, "SPH code)."
    for time, i_step in [(i*t_end/n_steps, i) for i in range(1, n_steps+1)]:
        hydro_legacy_code.evolve_model(time)
        if not i_step % 4:
            hydro_plot(
                view,
                hydro_legacy_code,
                (300, 300),
                base_output_file_name + "_hydro_image{0:=03}.png".format(i_step)
            )
    hydro_legacy_code.stop()
    print "All done!\n"


def hydro_plot(view, hydro_code, image_size, figname):
    """
    view: the (physical) region to plot [xmin, xmax, ymin, ymax]
    hydro_code: hydrodynamics code in which the gas to be plotted is defined
    image_size: size of the output image in pixels (x, y)
    """
    shape = (image_size[0], image_size[1], 1)
    size = image_size[0] * image_size[1]
    axis_lengths = [0.0, 0.0, 0.0] | units.m
    axis_lengths[0] = view[1] - view[0]
    axis_lengths[1] = view[3] - view[2]
    grid = Grid.create(shape, axis_lengths)
    grid.x += view[0]
    grid.y += view[2]
    speed = grid.z.reshape(size) * (0 | 1/units.s)
    rho, rhovx, rhovy, rhovz, rhoe = hydro_code.get_hydro_state_at_point(grid.x.reshape(size),
        grid.y.reshape(size), grid.z.reshape(size), speed, speed, speed)

    min_v =  800.0 | units.km / units.s
    max_v = 3000.0 | units.km / units.s
    min_rho = 3.0e-4 | units.g / units.cm**3
    max_rho = 0.1 | units.g / units.cm**3
    min_E = 1.0e11 | units.J / units.kg
    max_E = 1.0e13 | units.J / units.kg

    v_sqr = (rhovx**2 + rhovy**2 + rhovz**2) / rho**2
    E = rhoe / rho
    log_v = numpy.log((v_sqr / min_v**2)) / numpy.log((max_v**2 / min_v**2))
    log_rho = numpy.log((rho / min_rho)) / numpy.log((max_rho / min_rho))
    log_E = numpy.log((E / min_E)) / numpy.log((max_E / min_E))

    red   = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_rho)).reshape(shape)
    green = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_v)).reshape(shape)
    blue  = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_E)).reshape(shape)
    alpha = numpy.minimum(numpy.ones_like(log_v), numpy.maximum(numpy.zeros_like(log_v),
        numpy.log((rho / (10*min_rho))))).reshape(shape)

    rgba = numpy.concatenate((red, green, blue, alpha), axis = 2)

    pyplot.figure(figsize = (image_size[0]/100.0, image_size[1]/100.0), dpi = 100)
    im = pyplot.figimage(rgba, origin='lower')

    pyplot.savefig(figname, transparent=True, dpi = 100)
    print "\nHydroplot was saved to: ", figname
    pyplot.close()


if __name__  == "__main__":
    print "Running the simulation that formed the basis of the christmas card of Leiden Observatory of 2010."
    print
    print "Details:"
    print "The ornaments are the result of a smoothed particle simulation " \
        "with 50000 equal mass particles of a 310 Myr star of 0.3 solar mass, " \
        "which is ejected from a distance of 4 solar radii (left) " \
        "with a velocity of 3000 km/s into a 3.0 solar mass star at an " \
        "age of 310 Myr. The calculation was performed using the AMUSE " \
        "(amusecode.org) software environment in which the stars were evolved " \
        "using MESA to an age of 310 Myr before the encounter was performed " \
        "using Fi. Each ornament, generated using pyplot, is a snapshot from the " \
        "simulation, from top left to bottom right. The peak is created from a " \
        "blend of all snapshots. The colors of the ornaments are, red: log " \
        "of the density, green: log of the speed and for blue we used " \
        "the log of the specific internal energy."
    print
    if HAS_MATPLOTLIB:
        head_on_stellar_merger(number_of_sph_particles=500, sph_code=Gadget2)
    else:
        print "matplotlib is not installed. Install it in the site-packages folder of your Python installation. Returning."

Running the simulation that formed the basis of the christmas card of Leiden Observatory of 2010.

Details:
The ornaments are the result of a smoothed particle simulation with 50000 equal mass particles of a 310 Myr star of 0.3 solar mass, which is ejected from a distance of 4 solar radii (left) with a velocity of 3000 km/s into a 3.0 solar mass star at an age of 310 Myr. The calculation was performed using the AMUSE (amusecode.org) software environment in which the stars were evolved using MESA to an age of 310 Myr before the encounter was performed using Fi. Each ornament, generated using pyplot, is a snapshot from the simulation, from top left to bottom right. The peak is created from a blend of all snapshots. The colors of the ornaments are, red: log of the density, green: log of the speed and for blue we used the log of the specific internal energy.

Evolving stars with MESA...
Creating initial conditions from a MESA stellar evolution model:
0.299999104896 MSun star consisting of 

In [5]:
import os.path
from amuse.test.amusetest import get_path_to_results
try:
    from matplotlib import pyplot
    HAS_MATPLOTLIB = True
    from amuse.plot import plot, semilogy, xlabel, ylabel, loglog
except ImportError:
    HAS_MATPLOTLIB = False

from amuse.units import units
from amuse.units import constants
from amuse.units.generic_unit_converter import ConvertBetweenGenericAndSiUnits
from amuse.support.exceptions import AmuseException
from amuse.community.mesa.interface import MESA
from amuse.community.gadget2.interface import Gadget2
from amuse.community.fi.interface import Fi
from amuse.ext.star_to_sph import convert_stellar_model_to_SPH, pickle_stellar_model, StellarModel2SPH
from optparse import OptionParser
import numpy

from amuse.datamodel import Particles
from amuse.datamodel import ParticlesSuperset
from amuse.datamodel import Grid
def slowtest1():
    head_on_stellar_merger()

def head_on_stellar_merger(
        masses = [0.3, 3.0] | units.MSun,
        star_age = 310.0 | units.Myr,
        maximally_evolved_stars = False,
        initial_separation = 4.0 | units.RSun,
        angle = numpy.pi / 3,
        initial_speed = 3000.0 | units.km / units.s,
        initial_speed_perpendicular = 30.0 | units.km / units.s,
        number_of_sph_particles = 1000,
        t_end = 1.0e4 | units.s,
        sph_code = Fi,
        steps_per_snapshot = 4,
        snapshot_size = 100,
        use_stored_stellar_models = True
    ):
    """
    masses: Mass of the two stars
    star_age: Initial age of the stars (if maximally_evolved_stars is False)
    maximally_evolved_stars: Evolve stars as far as the Stellar Evolution code can get
    number_of_sph_particles: Total number of particles of both stars, divided according to their masses
    t_end: (Physical, not computational) duration of the hydrodynamics simulation
    sph_code: Code to use for the hydrodynamics simulation
    steps_per_snapshot: A hydroplot snapshot is generated each time after this many steps (0 or None means no snapshots)
    snapshot_size: Size of the snapshot in pixels along one dimension
    use_stored_stellar_models: Flag to use previously stored stellar model files (for speed-up).
    """

    # Convert some of the input parameters to string, for use in output file names:
    n_string = "n" + ("%1.0e"%(number_of_sph_particles)).replace("+0","").replace("+","")
    t_end_string = "t" + ("%1.0e"%(t_end.value_in(units.s))).replace("+0","").replace("+","")
    masses_string = ("m1_" + ("%0.3e"%(masses[0].value_in(units.MSun))).replace("+0","").replace("+","") +
        "_m2_" + ("%0.3e"%(masses[1].value_in(units.MSun))).replace("+0","").replace("+",""))
    if maximally_evolved_stars:
        star_age_string = "a_max"
    else:
        star_age_string = "a" + ("%0.3e"%(star_age.value_in(units.Myr))).replace("+0","").replace("+","")

    base_output_file_name = os.path.join(get_path_to_results(), "stellar_merger_"+n_string+"_"+t_end_string)
    pickle_file_1 = os.path.join(get_path_to_results(), "stellar_merger_"+masses_string+"_"+star_age_string+"_1.pkl")
    pickle_file_2 = os.path.join(get_path_to_results(), "stellar_merger_"+masses_string+"_"+star_age_string+"_2.pkl")

    if not use_stored_stellar_models or not (os.path.exists(pickle_file_1) and os.path.exists(pickle_file_2)):
        stars =  Particles(2)
        stars.mass = masses
        try:
            stellar_evolution = MESA(redirection='none', number_of_workers=4)
            stellar_evolution.initialize_code()
        except:
            print "MESA was not built. Returning."
            return
        stellar_evolution.commit_parameters()
        stellar_evolution.particles.add_particles(stars)
        stellar_evolution.commit_particles()

        if maximally_evolved_stars:
            try:
                while True:
                    stellar_evolution.evolve_model()
            except AmuseException as exception:
                print exception
        else:
            stellar_evolution.evolve_model(star_age)

        if os.path.exists(pickle_file_1):
            print "Could not save stellar model 1: file already exists."
        else:
            pickle_stellar_model(stellar_evolution.particles[0], pickle_file_1)
            print "Stellar model 1 saved at:", pickle_file_1
        if os.path.exists(pickle_file_2):
            print "Could not save stellar model 2: file already exists."
        else:
            pickle_stellar_model(stellar_evolution.particles[1], pickle_file_2)
            print "Stellar model 2 saved at:", pickle_file_2

        stellar_evolution.stop()

    model_1 = StellarModel2SPH(None, None, pickle_file = pickle_file_1)
    model_2 = StellarModel2SPH(None, None, pickle_file = pickle_file_2)
    model_1.unpickle_stellar_structure()
    model_2.unpickle_stellar_structure()
    composition = model_2.composition_profile
    midpoints = model_2.midpoints_profile[1:-1]
    specific_internal_energy = model_2.specific_internal_energy_profile

    number_of_sph_particles_1 = int(round(number_of_sph_particles *
        (model_1.mass / (model_1.mass + model_2.mass))))
    number_of_sph_particles_2 = number_of_sph_particles - number_of_sph_particles_1
    print "Creating initial conditions from a MESA stellar evolution model:"
    print model_1.mass, "star consisting of", number_of_sph_particles_1, "particles."
    sph_particles_1 = convert_stellar_model_to_SPH(
        None,
        number_of_sph_particles_1,
        seed=12345,
        pickle_file = pickle_file_1
    ).gas_particles
    print model_2.mass, "star consisting of", number_of_sph_particles_2, "particles."
    sph_particles_2 = convert_stellar_model_to_SPH(
        None,
        number_of_sph_particles_2,
        pickle_file = pickle_file_2
    ).gas_particles
    initial_separation += model_1.radius + model_2.radius
    sph_particles_2.x  += numpy.cos(angle) * initial_separation
    sph_particles_2.y  += numpy.sin(angle) * initial_separation
    sph_particles_1.vx += numpy.cos(angle) * initial_speed - numpy.sin(angle) * initial_speed_perpendicular
    sph_particles_1.vy += numpy.cos(angle) * initial_speed_perpendicular + numpy.sin(angle) * initial_speed
    view = [-0.5, 0.5, -0.5, 0.5] * (initial_separation + model_1.radius + model_2.radius)

    all_sph_particles = ParticlesSuperset([sph_particles_1, sph_particles_2])
    all_sph_particles.move_to_center()

    unit_converter = ConvertBetweenGenericAndSiUnits(1.0 | units.RSun, constants.G, t_end)
    hydro_legacy_code = sph_code(unit_converter, redirection='none', number_of_workers=4)
    n_steps = 100
    hydro_legacy_code.parameters.n_smooth = 96
    try:
        hydro_legacy_code.parameters.timestep = t_end / n_steps
    except Exception as exc:
        if not "parameter is read-only" in str(exc): raise
    hydro_legacy_code.gas_particles.add_particles(all_sph_particles)

    times = [] | units.Myr
    kinetic_energies =   [] | units.J
    potential_energies = [] | units.J
    thermal_energies =   [] | units.J

    print "Evolving to:", t_end
    for time, i_step in [(i*t_end/n_steps, i) for i in range(1, n_steps+1)]:
        hydro_legacy_code.evolve_model(time)
        times.append(time)
        kinetic_energies.append(   hydro_legacy_code.kinetic_energy)
        potential_energies.append( hydro_legacy_code.potential_energy)
        thermal_energies.append(   hydro_legacy_code.thermal_energy)
        if steps_per_snapshot and (not i_step % steps_per_snapshot):
            hydro_plot(
                view,
                hydro_legacy_code,
                (snapshot_size, snapshot_size),
                base_output_file_name + "_hydro_image{0:=03}.png".format(i_step)
            )


    hydro_legacy_code.gas_particles.new_channel_to(all_sph_particles).copy_attributes(
        ['mass', 'x','y','z', 'vx','vy','vz', 'u'])
    center_of_mass = all_sph_particles.center_of_mass().as_quantity_in(units.RSun)
    center_of_mass_velocity = all_sph_particles.center_of_mass_velocity().as_quantity_in(units.km / units.s)
    print
    print "center_of_mass:", center_of_mass
    print "center_of_mass_velocity:", center_of_mass_velocity
    all_sph_particles.position -= center_of_mass
    sph_midpoints = all_sph_particles.position.lengths()

    energy_plot(
        times,
        kinetic_energies, potential_energies, thermal_energies,
        base_output_file_name+"_energy_evolution.png"
    )
    thermal_energy_plot(
        times,
        thermal_energies,
        base_output_file_name+"_thermal_energy_evolution.png"
    )
    composition_comparison_plot(
        midpoints, composition[0],
        sph_midpoints, all_sph_particles.h1,
        base_output_file_name+"_composition_h1.png"
    )
    internal_energy_comparison_plot(
        midpoints, specific_internal_energy,
        sph_midpoints, all_sph_particles.u,
        base_output_file_name+"_new_u.png"
    )
    hydro_plot(
        [-2.0, 2.0, -2.0, 2.0] | units.RSun,
        hydro_legacy_code,
        (100, 100),
        base_output_file_name + "_hydro_image.png"
    )
    hydro_legacy_code.stop()
    print "All done!\n"


def composition_comparison_plot(radii_SE, comp_SE, radii_SPH, comp_SPH, figname):
    if not HAS_MATPLOTLIB:
        return
    pyplot.figure(figsize = (7, 5))
    plot(radii_SE.as_quantity_in(units.RSun), comp_SE,
        label='stellar evolution model')
    plot(radii_SPH, comp_SPH, 'go', label='SPH model')
    xlabel('radius')
    ylabel('mass fraction')
    pyplot.legend()
    pyplot.savefig(figname)
    print "\nPlot of composition profiles was saved to: ", figname
    pyplot.close()

def internal_energy_comparison_plot(radii_SE, u_SE, radii_SPH, u_SPH, figname):
    if not HAS_MATPLOTLIB:
        return
    pyplot.figure(figsize = (7, 5))
    semilogy(radii_SE.as_quantity_in(units.RSun), u_SE,
        label='stellar evolution model')
    semilogy(radii_SPH, u_SPH, 'go', label='SPH model')
    xlabel('radius')
    ylabel('internal energy')
    pyplot.legend()
    pyplot.savefig(figname)
    print "\nPlot of internal energy profiles was saved to: ", figname
    pyplot.close()

def energy_plot(time, E_kin, E_pot, E_therm, figname):
    if not HAS_MATPLOTLIB:
        return
    pyplot.figure(figsize = (5, 5))
    plot(time, E_kin.as_quantity_in(units.erg), label='E_kin')
    plot(time, E_pot, label='E_pot')
    plot(time, E_therm, label='E_therm')
    plot(time, E_kin+E_pot+E_therm, label='E_total')
    xlabel('Time')
    ylabel('Energy')
    pyplot.legend(loc=3)
    pyplot.savefig(figname)
    print "\nPlot of energy evolution was saved to: ", figname
    pyplot.close()

def thermal_energy_plot(time, E_therm, figname):
    if not HAS_MATPLOTLIB:
        return
    pyplot.figure(figsize = (5, 5))
    plot(time, E_therm.as_quantity_in(units.erg), label='E_therm')
    xlabel('Time')
    ylabel('Energy')
    pyplot.legend(loc=3)
    pyplot.savefig(figname)
    print "\nPlot of thermal energy evolution was saved to: ", figname
    pyplot.close()

def hydro_plot(view, hydro_code, image_size, figname):
    """
    view: the (physical) region to plot [xmin, xmax, ymin, ymax]
    hydro_code: hydrodynamics code in which the gas to be plotted is defined
    image_size: size of the output image in pixels (x, y)
    """
    if not HAS_MATPLOTLIB:
        return
    shape = (image_size[0], image_size[1], 1)
    size = image_size[0] * image_size[1]
    axis_lengths = [0.0, 0.0, 0.0] | units.m
    axis_lengths[0] = view[1] - view[0]
    axis_lengths[1] = view[3] - view[2]
    grid = Grid.create(shape, axis_lengths)
    grid.x += view[0]
    grid.y += view[2]
    speed = grid.z.reshape(size) * (0 | 1/units.s)
    rho, rhovx, rhovy, rhovz, rhoe = hydro_code.get_hydro_state_at_point(grid.x.reshape(size),
        grid.y.reshape(size), grid.z.reshape(size), speed, speed, speed)

    min_v =  800.0 | units.km / units.s
    max_v = 3000.0 | units.km / units.s
    min_rho = 3.0e-4 | units.g / units.cm**3
    max_rho = 0.1 | units.g / units.cm**3
    min_E = 1.0e11 | units.J / units.kg
    max_E = 1.0e13 | units.J / units.kg

    v_sqr = (rhovx**2 + rhovy**2 + rhovz**2) / rho**2
    E = rhoe / rho
    log_v = numpy.log((v_sqr / min_v**2)) / numpy.log((max_v**2 / min_v**2))
    log_rho = numpy.log((rho / min_rho)) / numpy.log((max_rho / min_rho))
    log_E = numpy.log((E / min_E)) / numpy.log((max_E / min_E))

    red   = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_rho)).reshape(shape)
    green = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_v)).reshape(shape)
    blue  = numpy.minimum(numpy.ones_like(rho.number), numpy.maximum(numpy.zeros_like(rho.number), log_E)).reshape(shape)
    alpha = numpy.minimum(numpy.ones_like(log_v), numpy.maximum(numpy.zeros_like(log_v),
        numpy.log((rho / (10*min_rho))))).reshape(shape)

    rgba = numpy.concatenate((red, green, blue, alpha), axis = 2)

    pyplot.figure(figsize = (image_size[0]/100.0, image_size[1]/100.0), dpi = 100)
    im = pyplot.figimage(rgba, origin='lower')

    pyplot.savefig(figname, transparent=True, dpi = 100)
    print "\nHydroplot was saved to: ", figname
    pyplot.close()



def new_option_parser():
    result = OptionParser()
    result.add_option(
        "-n",
        "--n_sph",
        dest="number_of_sph_particles",
        type="int",
        default = 300,
        help="Number of sph particles per star in solar mass"
    )
    result.add_option(
        "-a",
        "--mass1",
        dest="mass1",
        type="float",
        default = 1.0,
        help="Mass of the first star"
    )
    result.add_option(
        "-b",
        "--mass2",
        dest="mass2",
        type="float",
        default = 1.0,
        help="Mass of the second star in solar mass"
    )
    result.add_option(
        "-d",
        "--distance",
        dest="initial_separation",
        type="float",
        default=4.0,
        help="Initial_separation between the two stars in solar radii"
    )
    result.add_option(
        "-v",
        "--speed",
        dest="initial_speed",
        type="float",
        default=100.0,
        help="Initial relative speed between the two stars in km/s"
    )
    result.add_option(
        "-t",
        "--end_time",
        dest="end_time",
        type="float",
        default=2.0e4,
        help="Time to evolve to in s"
    )
    result.add_option(
        "-s",
        "--t_star",
        dest="t_star",
        type="float",
        default=1.0e4,
        help="Age of both stars in Myr"
    )
    result.add_option(
        "-e",
        "--max_evolved",
        default=False,
        action="store_true",
        dest="maximally_evolved_stars",
        help="Flag to use maximally evolved stars, i.e. until MESA exits"
    )
    return result


if __name__ == "__main__":
    options, arguments  = new_option_parser().parse_args()
    print "Simulating a head on collision between two stars."
    print "Options used:", options
    masses = [options.mass1, options.mass2] | units.MSun
    head_on_stellar_merger(
        masses = masses,
        star_age = options.t_star | units.Myr,
        initial_separation = options.initial_separation | units.RSun,
        initial_speed = options.initial_speed | units.km / units.s,
        number_of_sph_particles = options.number_of_sph_particles,
        t_end = options.end_time | units.s,
        maximally_evolved_stars = options.maximally_evolved_stars,
        sph_code = Gadget2
    )

Usage: __main__.py [options]

__main__.py: error: no such option: -f


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.
